## 處理缺失資料 (null, NaN, NA)
### 對於缺失資料慣例的取捨

* 使用遮罩(mask) : 布林陣列、資料中的其中一個位元
* 哨兵值(sentinel value) : -99999、Nan(Not a Number)是IEEE浮點數規中的一個特殊值

### None : Python 的缺失資料
* None 是一個 Python的物件，不能隨意使用在 Numpy / Pandas 的陣列中
* 資料型態是 object

In [1]:
import numpy as np
import pandas as pd

vals1 = np.array([1,None,3,4])
vals1

array([1, None, 3, 4], dtype=object)

In [4]:
for dtype in ['object','int']:
    print("dtype =",dtype)
    %timeit np.arange(1E6,dtype=dtype).sum()
    print()

dtype = object
68.9 ms ± 1.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
2.6 ms ± 75.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



In [6]:
#vals1.sum() # 會有錯

### NaN : 缺失的數值資料
* 特殊的浮點數

In [8]:
vals2 = np.array([1,np.nan,3,4])
vals2,vals2.dtype

(array([ 1., nan,  3.,  4.]), dtype('float64'))

In [10]:
# NaN 跟任何數字運算都會是 NaN
1 + np.nan, 0 * np.nan, 0/np.nan, vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan, nan, nan, nan)

In [11]:
# 忽略缺失值
np.nansum(vals2),np.nanmax(vals2),np.nanmin(vals2)

(8.0, 4.0, 1.0)

### 在 Pandas 中的 NaN 和 None
NaN 和 None 都有它們存在，而Pandas則是交換著使用他們，在他們之間適當地轉換:

In [12]:
pd.Series([1,np.nan,2,None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

對於那些沒有哨兵值的型態來說，Pandas會在NA值出現時自動轉型